<a href="https://colab.research.google.com/github/culiacanai/Aprende_Python_con_GoogleColab/blob/main/notebooks/12_Analisis_de_Datos_Real.ipynb" target="_parent">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# 📈 Análisis de Datos Real

### Aprende Python con Google Colab — por [Culiacan.AI](https://culiacan.ai)

**Nivel:** 🔴 Avanzado  
**Duración estimada:** 90 minutos  
**Requisitos:** Haber completado los Notebooks [07 Pandas](07_Pandas_Basico.ipynb), [08 Matplotlib](08_Visualizacion_con_Matplotlib.ipynb) y [11 NumPy](11_Intro_a_NumPy.ipynb)

---

En este notebook vas a:
- Trabajar con **datos reales** de fuentes internacionales (Gapminder, World Bank)
- Aplicar todo lo aprendido: Pandas, NumPy, Matplotlib en un flujo de análisis completo
- Hacer limpieza, transformación y análisis exploratorio (EDA) profesional
- Descubrir tendencias, correlaciones y patrones en datos de México y el mundo
- Crear visualizaciones con insights accionables
- Generar un reporte de hallazgos

> 💡 Este notebook integra todo lo que has aprendido en el curso. Es tu primer proyecto real de análisis de datos.

---

## Fuentes de datos

| Fuente | Qué contiene | URL |
|--------|-------------|-----|
| **Gapminder** | PIB per cápita, esperanza de vida y población de 142 países (1952-2007) | [gapminder.org](https://www.gapminder.org/data/) |
| **Plotly datasets** | Dataset Gapminder con códigos ISO | [github.com/plotly/datasets](https://github.com/plotly/datasets) |
| **World Bank API** | Indicadores de desarrollo por país | [data.worldbank.org](https://data.worldbank.org/) |


---

## 0. Preparación y carga de datos


In [ ]:
# Instalar plotly para acceder al dataset Gapminder integrado
!pip install plotly -q

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# --- Fuente 1: Gapminder vía Plotly (integrado, siempre funciona) ---
import plotly.express as px
df = px.data.gapminder()

print(f"✅ Dataset Gapminder cargado")
print(f"   Filas: {len(df):,}")
print(f"   Columnas: {list(df.columns)}")
print(f"   Países: {df['country'].nunique()}")
print(f"   Continentes: {df['continent'].unique()}")
print(f"   Período: {df['year'].min()} — {df['year'].max()}")

In [ ]:
# Exploración inicial
df.head(10)

In [ ]:
# Estadísticas generales
df.describe()

In [ ]:
# Información del DataFrame
df.info()

---

## 1. Limpieza y preparación de datos

El primer paso en cualquier análisis real es entender y limpiar los datos.


In [ ]:
# ¿Hay valores nulos?
print("Valores nulos por columna:")
print(df.isnull().sum())

# ¿Hay duplicados?
print(f"\nFilas duplicadas: {df.duplicated().sum()}")

# Verificar tipos de datos
print(f"\nTipos de datos:")
print(df.dtypes)

In [ ]:
# Renombrar columnas al español para mayor claridad
df = df.rename(columns={
    "country": "pais",
    "continent": "continente",
    "year": "anio",
    "lifeExp": "esperanza_vida",
    "pop": "poblacion",
    "gdpPercap": "pib_per_capita",
    "iso_alpha": "codigo_iso",
    "iso_num": "codigo_num",
})

# Traducir continentes
continentes_es = {
    "Asia": "Asia",
    "Europe": "Europa",
    "Africa": "África",
    "Americas": "Américas",
    "Oceania": "Oceanía",
}
df["continente"] = df["continente"].map(continentes_es)

# Crear columnas adicionales
df["pib_total"] = df["pib_per_capita"] * df["poblacion"]
df["decada"] = (df["anio"] // 10) * 10

print(f"✅ Datos preparados")
df.head()

---

## 2. Análisis Exploratorio Global

### 2.1 ¿Cómo ha evolucionado el mundo?


In [ ]:
# Evolución global por año
global_por_anio = df.groupby("anio").agg(
    esperanza_vida_media=("esperanza_vida", "mean"),
    pib_per_capita_medio=("pib_per_capita", "mean"),
    poblacion_total=("poblacion", "sum"),
    paises=("pais", "nunique"),
).reset_index()

fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Esperanza de vida
axes[0].plot(global_por_anio["anio"], global_por_anio["esperanza_vida_media"],
             marker="o", color="#2E86AB", linewidth=2.5)
axes[0].set_title("Esperanza de Vida Promedio", fontweight="bold")
axes[0].set_ylabel("Años")
axes[0].set_xlabel("Año")
axes[0].fill_between(global_por_anio["anio"], global_por_anio["esperanza_vida_media"],
                     alpha=0.1, color="#2E86AB")

# PIB per cápita
axes[1].plot(global_por_anio["anio"], global_por_anio["pib_per_capita_medio"],
             marker="s", color="#A23B72", linewidth=2.5)
axes[1].set_title("PIB per Cápita Promedio", fontweight="bold")
axes[1].set_ylabel("USD (ajustado)")
axes[1].set_xlabel("Año")
axes[1].yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f"${x:,.0f}"))

# Población
axes[2].plot(global_por_anio["anio"], global_por_anio["poblacion_total"] / 1e9,
             marker="D", color="#F18F01", linewidth=2.5)
axes[2].set_title("Población Mundial", fontweight="bold")
axes[2].set_ylabel("Miles de millones")
axes[2].set_xlabel("Año")

for ax in axes:
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.grid(axis="y", alpha=0.3)

plt.suptitle("Evolución Global (1952 — 2007)", fontsize=16, fontweight="bold")
plt.tight_layout()
plt.show()

### 2.2 Diferencias entre continentes


In [ ]:
# Esperanza de vida por continente a lo largo del tiempo
fig, ax = plt.subplots(figsize=(12, 6))

colores = {"África": "#C73E1D", "Américas": "#2E86AB", "Asia": "#F18F01",
           "Europa": "#A23B72", "Oceanía": "#27AE60"}

for continente in df["continente"].unique():
    datos = df[df["continente"] == continente].groupby("anio")["esperanza_vida"].mean()
    ax.plot(datos.index, datos.values, marker="o", markersize=4, linewidth=2.5,
            label=continente, color=colores.get(continente, "gray"))

ax.set_title("Esperanza de Vida por Continente (1952–2007)", fontsize=14, fontweight="bold")
ax.set_xlabel("Año")
ax.set_ylabel("Esperanza de vida (años)")
ax.legend(title="Continente", fontsize=10)
ax.grid(axis="y", alpha=0.3)
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)

plt.tight_layout()
plt.show()

In [ ]:
# Box plot: distribución por continente en 2007
datos_2007 = df[df["anio"] == 2007]

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Esperanza de vida
orden = datos_2007.groupby("continente")["esperanza_vida"].median().sort_values().index
datos_box = [datos_2007[datos_2007["continente"] == c]["esperanza_vida"].values for c in orden]

bp = axes[0].boxplot(datos_box, labels=orden, patch_artist=True,
                     medianprops={"color": "white", "linewidth": 2})
for patch, cont in zip(bp["boxes"], orden):
    patch.set_facecolor(colores.get(cont, "gray"))
    patch.set_alpha(0.8)
axes[0].set_title("Esperanza de Vida (2007)", fontweight="bold")
axes[0].set_ylabel("Años")

# PIB per cápita
datos_box_pib = [datos_2007[datos_2007["continente"] == c]["pib_per_capita"].values for c in orden]
bp2 = axes[1].boxplot(datos_box_pib, labels=orden, patch_artist=True,
                      medianprops={"color": "white", "linewidth": 2})
for patch, cont in zip(bp2["boxes"], orden):
    patch.set_facecolor(colores.get(cont, "gray"))
    patch.set_alpha(0.8)
axes[1].set_title("PIB per Cápita (2007)", fontweight="bold")
axes[1].set_ylabel("USD")
axes[1].yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f"${x:,.0f}"))

for ax in axes:
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)

plt.suptitle("Distribución por Continente — 2007", fontsize=14, fontweight="bold")
plt.tight_layout()
plt.show()

---

## 3. El gráfico de Hans Rosling

La visualización más famosa del mundo de datos: PIB per cápita vs esperanza de vida, con tamaño proporcional a la población. Inspirada en las presentaciones de [Hans Rosling de Gapminder](https://www.gapminder.org/).


In [ ]:
# Gráfico estilo Hans Rosling — año 2007
datos_2007 = df[df["anio"] == 2007].copy()

fig, ax = plt.subplots(figsize=(14, 9))

for continente in datos_2007["continente"].unique():
    subset = datos_2007[datos_2007["continente"] == continente]
    tamanio = (subset["poblacion"] / 1e6) ** 0.5 * 10  # Escalar para visibilidad

    ax.scatter(subset["pib_per_capita"], subset["esperanza_vida"],
               s=tamanio, alpha=0.65, label=continente,
               color=colores.get(continente, "gray"),
               edgecolors="white", linewidth=0.5)

# Anotar países destacados
paises_anotar = ["Mexico", "Brazil", "Chile", "Argentina", "United States",
                 "China", "India", "Japan", "Germany", "Nigeria",
                 "South Africa", "Cuba", "Colombia"]

for _, row in datos_2007.iterrows():
    if row["pais"] in paises_anotar:
        ax.annotate(row["pais"], (row["pib_per_capita"], row["esperanza_vida"]),
                    fontsize=8, ha="center", va="bottom",
                    textcoords="offset points", xytext=(0, 8))

ax.set_xscale("log")
ax.set_title("Salud vs Riqueza de las Naciones (2007)", fontsize=16, fontweight="bold")
ax.set_xlabel("PIB per cápita (USD, escala logarítmica)", fontsize=12)
ax.set_ylabel("Esperanza de vida (años)", fontsize=12)
ax.legend(title="Continente", fontsize=10, loc="lower right")
ax.grid(True, alpha=0.2)
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)

# Formato del eje X
ax.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f"${x:,.0f}"))

plt.tight_layout()
plt.show()

In [ ]:
# Comparar 1952 vs 2007
fig, axes = plt.subplots(1, 2, figsize=(16, 7))

for idx, anio in enumerate([1952, 2007]):
    ax = axes[idx]
    datos = df[df["anio"] == anio]

    for continente in datos["continente"].unique():
        subset = datos[datos["continente"] == continente]
        tamanio = (subset["poblacion"] / 1e6) ** 0.5 * 10

        ax.scatter(subset["pib_per_capita"], subset["esperanza_vida"],
                   s=tamanio, alpha=0.65, label=continente,
                   color=colores.get(continente, "gray"),
                   edgecolors="white", linewidth=0.5)

    ax.set_xscale("log")
    ax.set_title(f"{anio}", fontsize=14, fontweight="bold")
    ax.set_xlabel("PIB per cápita (USD)")
    ax.set_ylabel("Esperanza de vida (años)")
    ax.set_xlim(100, 60000)
    ax.set_ylim(25, 85)
    ax.grid(True, alpha=0.2)
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
    if idx == 1:
        ax.legend(title="Continente", fontsize=9, loc="lower right")

plt.suptitle("El Mundo en 1952 vs 2007", fontsize=16, fontweight="bold")
plt.tight_layout()
plt.show()

# Estadísticas del cambio
for continente in sorted(df["continente"].unique()):
    ev_1952 = df[(df["continente"] == continente) & (df["anio"] == 1952)]["esperanza_vida"].mean()
    ev_2007 = df[(df["continente"] == continente) & (df["anio"] == 2007)]["esperanza_vida"].mean()
    cambio = ev_2007 - ev_1952
    print(f"  {continente:<12} {ev_1952:.1f} → {ev_2007:.1f} años (+{cambio:.1f})")

---

## 4. Enfoque en México 🇲🇽

Vamos a analizar México en detalle y compararlo con otros países.


In [ ]:
# Datos de México
mexico = df[df["pais"] == "Mexico"].copy()

print("🇲🇽 MÉXICO — Evolución histórica")
print("=" * 60)
print(f"{'Año':>6} {'Esperanza vida':>16} {'Población':>14} {'PIB/cápita':>12}")
print("-" * 60)
for _, row in mexico.iterrows():
    print(f"{row['anio']:>6} {row['esperanza_vida']:>14.1f} años {row['poblacion']:>13,} ${row['pib_per_capita']:>11,.0f}")

# Crecimiento
ev_cambio = mexico.iloc[-1]["esperanza_vida"] - mexico.iloc[0]["esperanza_vida"]
pob_cambio = mexico.iloc[-1]["poblacion"] / mexico.iloc[0]["poblacion"]
pib_cambio = mexico.iloc[-1]["pib_per_capita"] / mexico.iloc[0]["pib_per_capita"]

print(f"\n📊 Cambios 1952 → 2007:")
print(f"  Esperanza de vida: +{ev_cambio:.1f} años")
print(f"  Población: ×{pob_cambio:.1f}")
print(f"  PIB per cápita: ×{pib_cambio:.1f}")

In [ ]:
# México vs países de referencia
paises_comp = ["Mexico", "Brazil", "Chile", "Argentina", "Colombia",
               "United States", "Cuba", "Costa Rica"]

fig, axes = plt.subplots(1, 3, figsize=(16, 5))

for pais in paises_comp:
    datos_pais = df[df["pais"] == pais]
    color = "#2E86AB" if pais == "Mexico" else "gray"
    alpha = 1 if pais == "Mexico" else 0.4
    lw = 3 if pais == "Mexico" else 1.5

    axes[0].plot(datos_pais["anio"], datos_pais["esperanza_vida"],
                 label=pais, color=color, alpha=alpha, linewidth=lw)
    axes[1].plot(datos_pais["anio"], datos_pais["pib_per_capita"],
                 label=pais, color=color, alpha=alpha, linewidth=lw)
    axes[2].plot(datos_pais["anio"], datos_pais["poblacion"] / 1e6,
                 label=pais, color=color, alpha=alpha, linewidth=lw)

axes[0].set_title("Esperanza de Vida", fontweight="bold")
axes[0].set_ylabel("Años")
axes[1].set_title("PIB per Cápita", fontweight="bold")
axes[1].set_ylabel("USD")
axes[1].yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f"${x:,.0f}"))
axes[2].set_title("Población", fontweight="bold")
axes[2].set_ylabel("Millones")

for ax in axes:
    ax.legend(fontsize=7, loc="upper left")
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.grid(axis="y", alpha=0.3)

plt.suptitle("México vs Países de Referencia", fontsize=14, fontweight="bold")
plt.tight_layout()
plt.show()

In [ ]:
# ¿Dónde se ubica México en 2007?
datos_2007 = df[df["anio"] == 2007].copy()

# Ranking por esperanza de vida
datos_2007["rank_ev"] = datos_2007["esperanza_vida"].rank(ascending=False).astype(int)
# Ranking por PIB per cápita
datos_2007["rank_pib"] = datos_2007["pib_per_capita"].rank(ascending=False).astype(int)

mx = datos_2007[datos_2007["pais"] == "Mexico"].iloc[0]
total_paises = len(datos_2007)

print(f"🇲🇽 Ranking de México en 2007 (de {total_paises} países):")
print(f"  Esperanza de vida: #{mx['rank_ev']:.0f} ({mx['esperanza_vida']:.1f} años)")
print(f"  PIB per cápita:    #{mx['rank_pib']:.0f} (${mx['pib_per_capita']:,.0f})")

# Top 10 y posición de México en esperanza de vida
print(f"\n🏆 Top 10 Esperanza de Vida (2007):")
top10_ev = datos_2007.nsmallest(10, "rank_ev")[["pais", "esperanza_vida", "rank_ev"]]
for _, row in top10_ev.iterrows():
    print(f"  #{row['rank_ev']:.0f} {row['pais']}: {row['esperanza_vida']:.1f} años")

# Vecinos de México en el ranking
print(f"\n📍 México y sus vecinos en el ranking:")
rank_mx = int(mx["rank_ev"])
vecinos = datos_2007[(datos_2007["rank_ev"] >= rank_mx - 3) &
                     (datos_2007["rank_ev"] <= rank_mx + 3)].sort_values("rank_ev")
for _, row in vecinos.iterrows():
    marcador = " 👈" if row["pais"] == "Mexico" else ""
    print(f"  #{row['rank_ev']:.0f} {row['pais']}: {row['esperanza_vida']:.1f} años{marcador}")

---

## 5. Correlaciones y análisis estadístico


In [ ]:
# Correlación entre PIB per cápita y esperanza de vida
datos_2007 = df[df["anio"] == 2007]

# Coeficiente de correlación
corr = datos_2007["pib_per_capita"].corr(datos_2007["esperanza_vida"])
corr_log = np.log10(datos_2007["pib_per_capita"]).corr(datos_2007["esperanza_vida"])

print(f"Correlación PIB vs Esperanza de vida: {corr:.3f}")
print(f"Correlación log(PIB) vs Esperanza de vida: {corr_log:.3f}")
print(f"\n💡 La relación es más fuerte con el logaritmo del PIB")
print(f"   Esto significa que duplicar el PIB importa más cuando eres pobre")

In [ ]:
# Matriz de correlación
datos_num = datos_2007[["esperanza_vida", "pib_per_capita", "poblacion"]].copy()
datos_num["log_pib"] = np.log10(datos_num["pib_per_capita"])
datos_num["log_pop"] = np.log10(datos_num["poblacion"])

corr_matrix = datos_num[["esperanza_vida", "log_pib", "log_pop"]].corr()

fig, ax = plt.subplots(figsize=(7, 6))

im = ax.imshow(corr_matrix, cmap="RdBu_r", vmin=-1, vmax=1)
labels = ["Esperanza\nde vida", "log(PIB\nper cápita)", "log(Población)"]

ax.set_xticks(range(len(labels)))
ax.set_yticks(range(len(labels)))
ax.set_xticklabels(labels, fontsize=10)
ax.set_yticklabels(labels, fontsize=10)

for i in range(len(labels)):
    for j in range(len(labels)):
        val = corr_matrix.iloc[i, j]
        color = "white" if abs(val) > 0.5 else "black"
        ax.text(j, i, f"{val:.2f}", ha="center", va="center",
                fontsize=14, fontweight="bold", color=color)

plt.colorbar(im, ax=ax, shrink=0.8)
ax.set_title("Matriz de Correlación (2007)", fontsize=14, fontweight="bold")
plt.tight_layout()
plt.show()

---

## 6. Análisis de desigualdad global


In [ ]:
# Evolución de la desigualdad (coeficiente de variación del PIB per cápita)
desigualdad = df.groupby("anio").apply(
    lambda x: pd.Series({
        "cv_pib": x["pib_per_capita"].std() / x["pib_per_capita"].mean(),
        "cv_ev": x["esperanza_vida"].std() / x["esperanza_vida"].mean(),
        "ratio_90_10_pib": np.percentile(x["pib_per_capita"], 90) / np.percentile(x["pib_per_capita"], 10),
        "brecha_ev": x["esperanza_vida"].max() - x["esperanza_vida"].min(),
    })
).reset_index()

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Ratio 90/10 del PIB
axes[0].plot(desigualdad["anio"], desigualdad["ratio_90_10_pib"],
             marker="o", color="#C73E1D", linewidth=2.5)
axes[0].set_title("Ratio PIB: País rico (P90) / País pobre (P10)", fontweight="bold")
axes[0].set_ylabel("Veces más rico")
axes[0].fill_between(desigualdad["anio"], desigualdad["ratio_90_10_pib"], alpha=0.1, color="#C73E1D")

# Brecha de esperanza de vida
axes[1].plot(desigualdad["anio"], desigualdad["brecha_ev"],
             marker="s", color="#2E86AB", linewidth=2.5)
axes[1].set_title("Brecha en Esperanza de Vida (máx - mín)", fontweight="bold")
axes[1].set_ylabel("Años de diferencia")
axes[1].fill_between(desigualdad["anio"], desigualdad["brecha_ev"], alpha=0.1, color="#2E86AB")

for ax in axes:
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.grid(axis="y", alpha=0.3)
    ax.set_xlabel("Año")

plt.suptitle("Evolución de la Desigualdad Global", fontsize=14, fontweight="bold")
plt.tight_layout()
plt.show()

print(f"📊 La desigualdad en PIB per cápita ha {'aumentado' if desigualdad.iloc[-1]['ratio_90_10_pib'] > desigualdad.iloc[0]['ratio_90_10_pib'] else 'disminuido'}")
print(f"   1952: P90 era {desigualdad.iloc[0]['ratio_90_10_pib']:.0f}x más rico que P10")
print(f"   2007: P90 era {desigualdad.iloc[-1]['ratio_90_10_pib']:.0f}x más rico que P10")

---

## 7. ¿Qué países mejoraron más?


In [ ]:
# Calcular mejora 1952 → 2007
datos_1952 = df[df["anio"] == 1952].set_index("pais")
datos_2007 = df[df["anio"] == 2007].set_index("pais")

paises_comunes = datos_1952.index.intersection(datos_2007.index)

mejora = pd.DataFrame({
    "pais": paises_comunes,
    "ev_1952": datos_1952.loc[paises_comunes, "esperanza_vida"].values,
    "ev_2007": datos_2007.loc[paises_comunes, "esperanza_vida"].values,
    "pib_1952": datos_1952.loc[paises_comunes, "pib_per_capita"].values,
    "pib_2007": datos_2007.loc[paises_comunes, "pib_per_capita"].values,
    "continente": datos_2007.loc[paises_comunes, "continente"].values,
})

mejora["mejora_ev"] = mejora["ev_2007"] - mejora["ev_1952"]
mejora["crecimiento_pib"] = ((mejora["pib_2007"] / mejora["pib_1952"]) - 1) * 100

# Top 10 mayor mejora en esperanza de vida
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Mayor mejora en esperanza de vida
top_ev = mejora.nlargest(15, "mejora_ev")
colors_ev = [colores.get(c, "gray") for c in top_ev["continente"]]
axes[0].barh(top_ev["pais"], top_ev["mejora_ev"], color=colors_ev)
axes[0].set_title("Mayor Mejora en Esperanza de Vida\n(1952 → 2007)", fontweight="bold")
axes[0].set_xlabel("Años ganados")
for i, (_, row) in enumerate(top_ev.iterrows()):
    axes[0].text(row["mejora_ev"] + 0.3, i, f"+{row['mejora_ev']:.1f}", va="center", fontsize=8)

# Menor mejora (o retroceso)
bottom_ev = mejora.nsmallest(15, "mejora_ev")
colors_bottom = [colores.get(c, "gray") for c in bottom_ev["continente"]]
axes[1].barh(bottom_ev["pais"], bottom_ev["mejora_ev"], color=colors_bottom)
axes[1].set_title("Menor Mejora en Esperanza de Vida\n(1952 → 2007)", fontweight="bold")
axes[1].set_xlabel("Años ganados")

for ax in axes:
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)

plt.tight_layout()
plt.show()

# ¿Dónde está México?
mx_mejora = mejora[mejora["pais"] == "Mexico"].iloc[0]
rank_mejora = mejora["mejora_ev"].rank(ascending=False)
mx_rank = rank_mejora[mejora["pais"] == "Mexico"].values[0]
print(f"\n🇲🇽 México: +{mx_mejora['mejora_ev']:.1f} años (#{mx_rank:.0f} de {len(mejora)} países)")

---

## 8. Enriquecer con datos del World Bank API

Vamos a complementar nuestro análisis con datos actualizados del Banco Mundial:


In [ ]:
import requests
import json

def obtener_indicador_wb(indicador: str, pais: str = "MX", anios: int = 30) -> pd.DataFrame:
    """
    Descarga un indicador del World Bank API.

    Indicadores útiles:
    - NY.GDP.PCAP.CD: PIB per cápita (USD actuales)
    - SP.DYN.LE00.IN: Esperanza de vida al nacer
    - SP.POP.TOTL: Población total
    - SI.POV.GINI: Índice GINI
    - SL.UEM.TOTL.ZS: Desempleo (% de fuerza laboral)
    """
    url = f"https://api.worldbank.org/v2/country/{pais}/indicator/{indicador}"
    params = {"format": "json", "per_page": anios}

    try:
        respuesta = requests.get(url, params=params, timeout=10)
        datos = respuesta.json()

        if len(datos) < 2 or datos[1] is None:
            print(f"⚠️ No hay datos para {indicador}")
            return pd.DataFrame()

        registros = []
        for d in datos[1]:
            if d["value"] is not None:
                registros.append({
                    "anio": int(d["date"]),
                    "valor": d["value"],
                    "pais": d["country"]["value"],
                })

        df = pd.DataFrame(registros).sort_values("anio")
        return df

    except Exception as e:
        print(f"⚠️ Error al consultar World Bank API: {e}")
        return pd.DataFrame()

# Descargar datos recientes de México
print("📥 Descargando datos del World Bank API...")

df_pib_mx = obtener_indicador_wb("NY.GDP.PCAP.CD", "MX", 30)
df_ev_mx = obtener_indicador_wb("SP.DYN.LE00.IN", "MX", 30)
df_pop_mx = obtener_indicador_wb("SP.POP.TOTL", "MX", 30)
df_desempleo_mx = obtener_indicador_wb("SL.UEM.TOTL.ZS", "MX", 30)

print(f"✅ PIB per cápita: {len(df_pib_mx)} años")
print(f"✅ Esperanza de vida: {len(df_ev_mx)} años")
print(f"✅ Población: {len(df_pop_mx)} años")
print(f"✅ Desempleo: {len(df_desempleo_mx)} años")

In [ ]:
# Visualizar datos actualizados de México
if not df_pib_mx.empty and not df_ev_mx.empty:
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))

    # PIB per cápita
    if not df_pib_mx.empty:
        axes[0, 0].plot(df_pib_mx["anio"], df_pib_mx["valor"], marker="o",
                         color="#2E86AB", linewidth=2)
        axes[0, 0].set_title("PIB per Cápita (USD)", fontweight="bold")
        axes[0, 0].yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f"${x:,.0f}"))
        axes[0, 0].fill_between(df_pib_mx["anio"], df_pib_mx["valor"], alpha=0.1, color="#2E86AB")

    # Esperanza de vida
    if not df_ev_mx.empty:
        axes[0, 1].plot(df_ev_mx["anio"], df_ev_mx["valor"], marker="o",
                         color="#27AE60", linewidth=2)
        axes[0, 1].set_title("Esperanza de Vida (años)", fontweight="bold")
        axes[0, 1].fill_between(df_ev_mx["anio"], df_ev_mx["valor"], alpha=0.1, color="#27AE60")

    # Población
    if not df_pop_mx.empty:
        axes[1, 0].plot(df_pop_mx["anio"], df_pop_mx["valor"] / 1e6, marker="o",
                         color="#F18F01", linewidth=2)
        axes[1, 0].set_title("Población (millones)", fontweight="bold")
        axes[1, 0].fill_between(df_pop_mx["anio"], df_pop_mx["valor"] / 1e6, alpha=0.1, color="#F18F01")

    # Desempleo
    if not df_desempleo_mx.empty:
        axes[1, 1].plot(df_desempleo_mx["anio"], df_desempleo_mx["valor"], marker="o",
                         color="#C73E1D", linewidth=2)
        axes[1, 1].set_title("Desempleo (%)", fontweight="bold")
        axes[1, 1].fill_between(df_desempleo_mx["anio"], df_desempleo_mx["valor"],
                                 alpha=0.1, color="#C73E1D")

    for ax in axes.flat:
        ax.spines["top"].set_visible(False)
        ax.spines["right"].set_visible(False)
        ax.grid(axis="y", alpha=0.3)

    plt.suptitle("🇲🇽 México — Indicadores Recientes (World Bank)",
                 fontsize=16, fontweight="bold")
    plt.tight_layout()
    plt.show()
else:
    print("⚠️ No se pudieron obtener datos del World Bank. Verifica tu conexión a internet.")

---

## 9. 🏆 Resumen de hallazgos

Vamos a compilar los insights más importantes de nuestro análisis:


In [ ]:
# Resumen automático de hallazgos
datos_2007 = df[df["anio"] == 2007]
datos_1952 = df[df["anio"] == 1952]

print("=" * 65)
print("  📊 RESUMEN DE HALLAZGOS — ANÁLISIS DE DATOS REAL")
print("  Dataset: Gapminder + World Bank | Culiacan.AI")
print("=" * 65)

# Hallazgo 1: Progreso global
ev_global_1952 = datos_1952["esperanza_vida"].mean()
ev_global_2007 = datos_2007["esperanza_vida"].mean()
print(f"\n1. 🌍 PROGRESO GLOBAL")
print(f"   La esperanza de vida mundial pasó de {ev_global_1952:.1f} a {ev_global_2007:.1f} años")
print(f"   Un aumento de +{ev_global_2007 - ev_global_1952:.1f} años en 55 años")

# Hallazgo 2: Desigualdad persistente
ev_max = datos_2007["esperanza_vida"].max()
ev_min = datos_2007["esperanza_vida"].min()
pais_max = datos_2007.loc[datos_2007["esperanza_vida"].idxmax(), "pais"]
pais_min = datos_2007.loc[datos_2007["esperanza_vida"].idxmin(), "pais"]
print(f"\n2. 📉 DESIGUALDAD PERSISTENTE")
print(f"   En 2007, hay {ev_max - ev_min:.1f} años de diferencia entre")
print(f"   {pais_max} ({ev_max:.1f} años) y {pais_min} ({ev_min:.1f} años)")

# Hallazgo 3: México
mx_2007 = datos_2007[datos_2007["pais"] == "Mexico"].iloc[0]
print(f"\n3. 🇲🇽 MÉXICO")
print(f"   Esperanza de vida: {mx_2007['esperanza_vida']:.1f} años")
print(f"   PIB per cápita: ${mx_2007['pib_per_capita']:,.0f}")
print(f"   Mejoró +{mx_mejora['mejora_ev']:.1f} años desde 1952")

# Hallazgo 4: Correlación
print(f"\n4. 💰 RELACIÓN SALUD-RIQUEZA")
print(f"   Correlación PIB vs Esperanza de vida: {corr:.3f}")
print(f"   El dinero importa, pero no es todo:")
print(f"   Cuba (${datos_2007[datos_2007['pais']=='Cuba']['pib_per_capita'].values[0]:,.0f}) tiene mayor esperanza de vida")
print(f"   que Sudáfrica (${datos_2007[datos_2007['pais']=='South Africa']['pib_per_capita'].values[0]:,.0f})")

# Hallazgo 5: Mayor crecimiento
top_crec = mejora.nlargest(1, "mejora_ev").iloc[0]
print(f"\n5. 🚀 MAYOR TRANSFORMACIÓN")
print(f"   {top_crec['pais']}: +{top_crec['mejora_ev']:.1f} años en esperanza de vida")
print(f"   De {top_crec['ev_1952']:.1f} a {top_crec['ev_2007']:.1f} años")

print(f"\n{'=' * 65}")
print("  Análisis realizado con Python + Pandas + Matplotlib + NumPy")
print("  Datos: Gapminder (CC-BY) + World Bank (Open Data)")
print(f"{'=' * 65}")

---

## 🔥 Retos

1. **Análisis por década:** Agrupa los datos por década y continente. Encuentra qué década tuvo el mayor avance en esperanza de vida para cada continente. Genera un heatmap de mejora por década × continente.

2. **Predicción simple:** Usando los datos históricos de México, calcula la tasa de crecimiento promedio anual del PIB per cápita y proyecta dónde estará en 2025, 2030 y 2050. Compara con la tendencia de Chile y Brasil.

3. **Análisis completo de otro país:** Elige un país que te interese y haz un análisis similar al de México: evolución histórica, ranking mundial, comparación con vecinos, y hallazgos clave. Genera un reporte visual de una página.


In [ ]:
# Reto 1: Análisis por década
# Tu código aquí 👇


In [ ]:
# Reto 2: Predicción simple
# Tu código aquí 👇


In [ ]:
# Reto 3: Análisis de otro país
# Tu código aquí 👇


---

## 📋 Resumen del proceso de análisis

| Paso | Qué hicimos | Herramientas |
|------|------------|-------------|
| 1. Obtener datos | Gapminder (plotly), World Bank API | `px.data.gapminder()`, `requests` |
| 2. Limpiar | Renombrar, traducir, crear columnas | Pandas |
| 3. Explorar (EDA) | describe, value_counts, info | Pandas |
| 4. Analizar tendencias | Evolución temporal, comparativas | Pandas + Matplotlib |
| 5. Visualizar | Scatter, líneas, barras, box plots, heatmap | Matplotlib |
| 6. Correlaciones | Matriz de correlación | NumPy + Pandas |
| 7. Comparar | México vs otros países, rankings | Pandas |
| 8. Enriquecer | Datos adicionales del World Bank | requests + API |
| 9. Comunicar | Resumen de hallazgos | Print formateado |

---

## ⏭️ ¿Qué sigue?

En el siguiente notebook damos el salto a **Machine Learning con Scikit-learn** — tu primer modelo de IA.

👉 [13 — Intro a Machine Learning](13_Intro_a_Machine_Learning.ipynb)

---

<p align="center">
  Hecho con ❤️ por <a href="https://culiacan.ai">Culiacan.AI</a> — Culiacán reconocida en el mundo por su talento y emprendimiento en Inteligencia Artificial
</p>
